[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/openai/openai-cookbook/blob/main/articles/gpt-oss/run-colab.ipynb)

# 無料のGoogle ColabでOpenAI gpt-oss 20Bを実行する

OpenAIは`gpt-oss` [120B](https://hf.co/openai/gpt-oss-120b)と[20B](https://hf.co/openai/gpt-oss-20b)をリリースしました。両方のモデルはApache 2.0ライセンスです。

特に、`gpt-oss-20b`は低レイテンシとローカルまたは特殊な用途向けに作られました（210億パラメータで36億のアクティブパラメータ）。

モデルはネイティブMXFP4量子化で訓練されているため、Google Colabのようなリソース制約のある環境でも20Bを簡単に実行できます。

著者：[Pedro](https://huggingface.co/pcuenq)と[VB](https://huggingface.co/reach-vb)

## 環境のセットアップ

transformersでのmxfp4サポートは最先端の機能であるため、`mxfp4` tritonカーネルをインストールできるように、PyTorchとCUDAの最新バージョンが必要です。

また、transformersをソースからインストールする必要があり、依存関係の競合を解決するために`torchvision`と`torchaudio`をアンインストールします。

In [ ]:
!pip install -q --upgrade torch

In [ ]:
!pip install -q transformers triton==3.4 kernels

In [ ]:
!pip uninstall -q torchvision torchaudio -y

上記のパッケージをインストールした後、Colabランタイムセッションを再起動してください。

## Google ColabでHugging Faceからモデルを読み込む

こちらからモデルを読み込みます：[openai/gpt-oss-20b](https://hf.co/openai/gpt-oss-20b)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "openai/gpt-oss-20b"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map="cuda",
)

## メッセージ/チャットの設定

オプションのシステムプロンプトを提供するか、直接入力を行うことができます。

In [ ]:
messages = [
    {"role": "system", "content": "Always respond in riddles"},
    {"role": "user", "content": "What is the weather like in Madrid?"},
]

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
    return_dict=True,
).to(model.device)

generated = model.generate(**inputs, max_new_tokens=500)
print(tokenizer.decode(generated[0][inputs["input_ids"].shape[-1]:]))

## 推論の努力レベルを指定する

`apply_chat_template()`の追加引数として渡すだけです。サポートされている値は`"low"`、`"medium"`（デフォルト）、または`"high"`です。

In [ ]:
messages = [
    {"role": "system", "content": "Always respond in riddles"},
    {"role": "user", "content": "Explain why the meaning of life is 42"},
]

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
    return_dict=True,
    reasoning_effort="high",
).to(model.device)

generated = model.generate(**inputs, max_new_tokens=500)
print(tokenizer.decode(generated[0][inputs["input_ids"].shape[-1]:]))

## 他のプロンプトやアイデアを試してみましょう！

他のアイデアについては、私たちのブログ投稿をご覧ください：[https://hf.co/blog/welcome-openai-gpt-oss](https://hf.co/blog/welcome-openai-gpt-oss)